<a href="https://colab.research.google.com/github/All4Nothing/pytorch-DL-project/blob/main/Ch03_Deep_CNN_Architecture(7).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 03. 심층 CNN 아키텍처

## EfficientNet과 CNN 아키텍처의 미래

CNN 모델을 확장 또는 스케일링 하는 방법  
* 계층 수 증가 - **깊이**
* 합성곱 계층에서 특징 맵 또는 채널 수 증가 - **너비**
* LeNet의 32 x 32 픽셀 이미지를 AlexNet의 224 x 224 픽셀 이미지로 공간 차원을 증가 - **해상도**

모델을 스케일링 하는 이 세가지 관점을 각각 **깊이, 너비, 해상도**라 한다. EfficientNet은 이 세가지 속성을 수동으로 조정하는 대신, 신경망 아키텍처를 검색함으로써 각 아키텍처에 대한 최적의 스케일링 계수를 계산한다.  

네트워크가 깊어질 수록 모델은 더 복잡해지고 그에 따라 상당히 복잡한 특징을 학습할 수 있으므로 깊이를 깊게 만드는 것이 중요한다. 그러나 깊이 증가함에 따라 경사 소실 문제가 과적합의 일반적인 문제와 함께 확대되므로 절충점을 찾아야 한다.  

마찬가지로 채널 수가 많아질수록 네트워크가 더 세밀한 특징을 학습할 수 있으므로 이론적으로는 너비를 증가시키는 것이 도움이 된다. 그렇지만 모델이 극도로 넓어지면 정확도가 빠르게 saturate(포화)되는 경향이 있다.

마지막으로 고행사도 이미지는 더 세분화된 정보를 포함하므로 이론적으로 더 잘 작동해야 한다. 그러나 경험적으로 해상도가 높아진 모델 성능이 동일한 수준으로 선형적으로 증가하는 것은 아니다. 이로써 스케일링 인자를 결정할 때 이뤄져야 할 트레이드오프가 있고 따라서 신경망 아키텍처 검색은 최적의 스케일링 인자를 찾는데 도움이 된다.

EfficientNet은 깊이, 너비, 해상도 사이에 적절한 균형을 갖는 아키텍처를 찾는 방법을 제안한다. 이 세가지 관점은 전역 스케일링 인자를 사용해 함께 스케일링 된다. EfficientNet 아키텍처는 두 단계로 구성된다. 첫 번째 단계에서 스케일링 인자를 1로 고정해 기본 아키텍처(기본 네트워크)를 만든다. 이 단계에서 주어진 작업과 데이터셋에 대해 길이, 너비, 해상도의 상대적 중요도가 결정된다.  
기본 네트워크는 유명한 CNN 아키텍처인 MnasNet(Mobile Neural Architecture Search Network)과 매우 유사한 방법으로 얻는다. 파이토치는 사전 훈련된 MnasNet 모델을 제공하며, 다음 코드를 사용해 로딩할 수 있다.

In [1]:
import torchvision.models as models
model = models.mnasnet1_0()

첫 번째 단계에서 기본 네트워크를 얻게 되면 모델 정확도를 최대화하고 계산(또는 실패)수를 최소화하는 최적의 전역 스케일링 인자가 계산된다. 기본 네트워크를 EfficientNet B0라고 하며, 다양한 최적 스케일링 인자로부터 유래된 후속 네트워크를 **EfficientNet B1~B7**이라고 한다.

앞으로 CNN 아키텍처를 효율적으로 확장하는 방법은 inception, 잔차, 밀집 모듈에서 영감을 받아 더욱 정교한 모듈을 개방하는 방법과 함께 중요한 연구 방향으로 자리 잡을 것이다. CNN 아키텍처에서 고려해야 할 또 다른 측면은 성능을 유지하면서 모델 크기를 최소화 하는 것이다. **MobileNets**가 대표적인 예로 이 분야에서 많은 연구가 진행중이다.

기존 모델의 아키텍처를 수정하는 방식 외에, CNN을 구성하는 요소(합성곱 커널, 풀링 메커니즘, 좀 더 효율적인 평면화 방식 등)를 근본적으로 개선하려는 노력도 이어졌다. 구체적인 예로는 이미지의 세 번째 차원(깊이)에 맞게 합성곱 구성 단위를 개조한 **CapsuleNet**을 들 수 있다.

ResNets를 객체 감지 및 분할에 **RCNN(Region Based Convolutional Neural Networks)** 형태로 사용할 수도 있다. RCNN을 개선한 변형으로는 **Faster R-CNN, Mask-RCNN, Keypoint-RCNN**을 들 수 있다.

In [2]:
faster_rcnn = models.detection.fasterrcnn_resnet50_fpn()
mask_rcnn = models.detection.maskrcnn_resnet50_fpn()
keypoint_rcnn = models.detection.keypointrcnn_resnet50_fpn()

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 104MB/s]


또한 파이토치는 동영상 분류 같은 동영상과 관련한 작업에 적용되는 ResNet의 사전 훈련된 모델도 제공한다. 이처럼 동영상 분류에 사용되는 ResNet 기반의 모델로는 **ResNet3D**과 **ResNet Mixed Convolution**이 있다.

In [3]:
resnet_3d = models.video.r3d_18()
resnet_mixed_conv = models.video.mc3_18()

https://pytorch.org/vision/stable/models.html#object-detection-instance-segmentation-and-person-keypoint-detection